### data collection

In [107]:
# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from itertools import chain
from tqdm import tqdm,tqdm_notebook
import random
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


DATA_PATH = Path('D:/ssd')

# read failed label from failed_data
df_failure = pd.read_csv(DATA_PATH / 'ssd_failure_label.csv',parse_dates=['failure_time']).sort_values(by='failure_time')
df_failure['date'] = df_failure.failure_time.map(lambda dt:dt.strftime('%Y%m%d'))
df_failure

,model,failure_time,disk_id,date
309,MA2,2018-01-02 19:15:32,33722,20180102
293,MA2,2018-01-02 22:45:16,58337,20180102
2197,MA1,2018-01-03 03:23:44,26378,20180103
2001,MA1,2018-01-03 03:29:27,39876,20180103
13424,MC1,2018-01-03 05:03:03,199348,20180103
...,...,...,...,...
16226,MC2,2019-12-31 16:16:05,12463,20191231
16066,MC2,2019-12-31 19:28:04,10186,20191231
13278,MC1,2019-12-31 19:32:50,18144,20191231
602,MA2,2019-12-31 21:57:36,99516,20191231


In [110]:
# read all failed labels from 2018.10.1 to 2018.12.31,  1633 rows
use_time_min,use_time_max = datetime(2018,10,1),datetime(2018,12,31)
failure_in_use_time_mask = (use_time_min< df_failure.failure_time) * (df_failure.failure_time<=use_time_max)
df_failure_use = df_failure[failure_in_use_time_mask]
failure_items = set([tuple(i) for i in df_failure_use[['disk_id','model']].values])
num_failure_items = len(failure_items)
df_failure_use

,model,failure_time,disk_id,date
9220,MC1,2018-10-01 00:14:45,175084,20181001
12140,MC1,2018-10-01 01:35:13,5922,20181001
1486,MA1,2018-10-01 03:07:53,41499,20181001
379,MA2,2018-10-01 03:14:46,58115,20181001
1417,MA1,2018-10-01 03:20:28,8964,20181001
...,...,...,...,...
13014,MC1,2018-12-30 17:51:31,40557,20181230
12909,MC1,2018-12-30 17:51:31,40544,20181230
12982,MC1,2018-12-30 21:08:03,843,20181230
2643,MB2,2018-12-30 21:25:25,22883,20181230


In [111]:
df_failure_use['model'].value_counts()

MC1    819
MC2    318
MA1    170
MB1    163
MA2     94
MB2     69
Name: model, dtype: int64

In [112]:
# Read data from 2018-10-1 and randomly sample num_failure_item from all error free labels
df_2018_7_1 = pd.read_csv(DATA_PATH/'smartlog2018ssd'/'20181001.csv')

all_items = set([tuple(i) for i in df_2018_7_1[['disk_id','model']].values])
no_failure_items_all = all_items - failure_items

# randomly sample
random.seed(2023)
no_failure_items = random.sample(no_failure_items_all, num_failure_items)
no_failure_items[:10]

[(78273, 'MA2'),
 (8378, 'MB1'),
 (27384, 'MB1'),
 (9504, 'MB2'),
 (86139, 'MC1'),
 (36671, 'MA2'),
 (8646, 'MB2'),
 (79431, 'MA2'),
 (119189, 'MA2'),
 (31685, 'MB2')]

In [113]:
# merge failure_items and no_failure_items
use_items = set(no_failure_items) | failure_items
len(use_items)

3266

In [114]:
# read data from 7.1 to 12.31， select rows contain no_failure_items and failure_items， combine into original data
read_data = sorted([fp for fp in (DATA_PATH/'smartlog2018ssd').rglob("*.csv") if ('20181001'<=fp.name.strip('.csv')<='20181231')])

all_use_df = []
for fp in tqdm_notebook(read_data,desc="filter data..."):
    df_tmp = pd.read_csv(fp)
    # find used rows
    use_items_mask = [(item[0],item[1]) in use_items for item in df_tmp[['disk_id','model']].values]
    df_tmp = df_tmp[use_items_mask]
    all_use_df.append(df_tmp.copy())
    print(fp, df_tmp.shape)

filter data...:   0%|          | 0/88 [00:00<?, ?it/s]

D:\ssd\smartlog2018ssd\20181001.csv (3038, 105)
D:\ssd\smartlog2018ssd\20181002.csv (2960, 105)
D:\ssd\smartlog2018ssd\20181003.csv (2955, 105)
D:\ssd\smartlog2018ssd\20181004.csv (2118, 105)
D:\ssd\smartlog2018ssd\20181005.csv (2919, 105)
D:\ssd\smartlog2018ssd\20181006.csv (2876, 105)
D:\ssd\smartlog2018ssd\20181007.csv (2943, 105)
D:\ssd\smartlog2018ssd\20181008.csv (3076, 105)
D:\ssd\smartlog2018ssd\20181009.csv (3059, 105)
D:\ssd\smartlog2018ssd\20181010.csv (3025, 105)
D:\ssd\smartlog2018ssd\20181011.csv (3048, 105)
D:\ssd\smartlog2018ssd\20181012.csv (3084, 105)
D:\ssd\smartlog2018ssd\20181013.csv (3083, 105)
D:\ssd\smartlog2018ssd\20181014.csv (3063, 105)
D:\ssd\smartlog2018ssd\20181015.csv (3046, 105)
D:\ssd\smartlog2018ssd\20181016.csv (3071, 105)
D:\ssd\smartlog2018ssd\20181017.csv (3056, 105)
D:\ssd\smartlog2018ssd\20181018.csv (3041, 105)
D:\ssd\smartlog2018ssd\20181019.csv (2936, 105)
D:\ssd\smartlog2018ssd\20181020.csv (3004, 105)
D:\ssd\smartlog2018ssd\20181021.csv (302

In [118]:
# merge all data
df_use = pd.concat(all_use_df).sort_values(by=['model','disk_id','ds'])
# df_use = pd.concat(all_use_df).sort_values(by=['ds', 'disk_id', 'model'])

In [119]:
df_use

,disk_id,ds,model,n_1,r_1,n_2,r_2,n_3,r_3,n_4,...,n_242,r_242,n_244,r_244,n_245,r_245,n_175,r_175,n_232,r_232
10918,406,20181001,MA1,130.0,4.137765e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,9.109222e+11,NaN,NaN
345397,406,20181002,MA1,130.0,4.175147e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,9.110161e+11,NaN,NaN
80313,406,20181003,MA1,130.0,4.215511e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,9.111100e+11,NaN,NaN
242442,406,20181004,MA1,130.0,4.254417e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,9.148383e+11,NaN,NaN
114364,406,20181005,MA1,130.0,4.293503e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,9.149322e+11,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323799,23851,20181227,MC2,100.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175712,23851,20181228,MC2,100.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344020,23851,20181229,MC2,100.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26818,23851,20181230,MC2,100.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
df_use[ : round(df_use.shape[0] * 0.3)]['model'].value_counts()

MA2    42547
MA1    19345
MB1    10822
Name: model, dtype: int64

In [121]:
# label the original data with failure: The rules for labeling are
# starting from the date of failure, all data with failure_label_range days of retrogression are labeled as 1
# while the rest are labeled as 0
# failure_label_range = 30

from datetime import timedelta

df_use['label'] = 'no'
for irow in tqdm_notebook(range(len(df_failure_use)), desc='label raw data...'):
    model,dt,disk_id,_ = df_failure_use.iloc[irow].values
    label1_min = int((dt - timedelta(days=29)).strftime('%Y%m%d'))
    label1_max = int(dt.strftime('%Y%m%d'))

    label_1_mask = (df_use.model==model) * (df_use.disk_id==disk_id) * (df_use.ds>=label1_min) * (df_use.ds<=label1_max)
    df_use.loc[label_1_mask,'label'] = 'yes'
    
    

label raw data...:   0%|          | 0/1633 [00:00<?, ?it/s]

In [122]:
df_use

,disk_id,ds,model,n_1,r_1,n_2,r_2,n_3,r_3,n_4,...,r_242,n_244,r_244,n_245,r_245,n_175,r_175,n_232,r_232,label
10918,406,20181001,MA1,130.0,4.137765e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,9.109222e+11,NaN,NaN,no
345397,406,20181002,MA1,130.0,4.175147e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,9.110161e+11,NaN,NaN,no
80313,406,20181003,MA1,130.0,4.215511e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,9.111100e+11,NaN,NaN,no
242442,406,20181004,MA1,130.0,4.254417e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,9.148383e+11,NaN,NaN,no
114364,406,20181005,MA1,130.0,4.293503e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,9.149322e+11,NaN,NaN,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323799,23851,20181227,MC2,100.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
175712,23851,20181228,MC2,100.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
344020,23851,20181229,MC2,100.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
26818,23851,20181230,MC2,100.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no


In [123]:
# save raw data
df_use.to_csv('D:/ssd/df_use.csv',index=None)
df_use

,disk_id,ds,model,n_1,r_1,n_2,r_2,n_3,r_3,n_4,...,r_242,n_244,r_244,n_245,r_245,n_175,r_175,n_232,r_232,label
10918,406,20181001,MA1,130.0,4.137765e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,9.109222e+11,NaN,NaN,no
345397,406,20181002,MA1,130.0,4.175147e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,9.110161e+11,NaN,NaN,no
80313,406,20181003,MA1,130.0,4.215511e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,9.111100e+11,NaN,NaN,no
242442,406,20181004,MA1,130.0,4.254417e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,9.148383e+11,NaN,NaN,no
114364,406,20181005,MA1,130.0,4.293503e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,9.149322e+11,NaN,NaN,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323799,23851,20181227,MC2,100.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
175712,23851,20181228,MC2,100.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
344020,23851,20181229,MC2,100.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
26818,23851,20181230,MC2,100.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no


In [124]:
# the label ratio of 0 to 1 after marking is 5:1
df_use.label.value_counts()

no     205821
yes     36560
Name: label, dtype: int64

## feature

In [125]:
# load checkpoint
df_clean = df_use.copy()
df_clean['ds']=df_clean['ds'].map(lambda x:datetime.strptime(str(x),'%Y%m%d'))

# observe that some features in the data are all empty values, delete these features
df_clean.info(3, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242381 entries, 10918 to 63450
Data columns (total 106 columns):
 #    Column   Non-Null Count   Dtype         
---   ------   --------------   -----         
 0    disk_id  242381 non-null  int64         
 1    ds       242381 non-null  datetime64[ns]
 2    model    242381 non-null  object        
 3    n_1      155937 non-null  float64       
 4    r_1      155937 non-null  float64       
 5    n_2      0 non-null       float64       
 6    r_2      0 non-null       float64       
 7    n_3      0 non-null       float64       
 8    r_3      0 non-null       float64       
 9    n_4      0 non-null       float64       
 10   r_4      0 non-null       float64       
 11   n_5      242361 non-null  float64       
 12   r_5      242361 non-null  float64       
 13   n_6      0 non-null       float64       
 14   r_6      0 non-null       float64       
 15   n_7      0 non-null       float64       
 16   r_7      0 non-null       float64

In [126]:
# some features have a variance of 0 (all are the same value, without discrimination, and need to be deleted)
df_clean_desc = df_clean.loc[:, 'n_1':'r_232'].describe()
df_clean_desc

,n_1,r_1,n_2,r_2,n_3,r_3,n_4,r_4,n_5,r_5,...,n_242,r_242,n_244,r_244,n_245,r_245,n_175,r_175,n_232,r_232
count,155937.000000,1.559370e+05,0.0,0.0,0.0,0.0,0.0,0.0,242361.000000,242361.000000,...,86422.000000,8.642200e+04,43883.0,43883.0,43881.0,43881.0,61887.000000,6.188700e+04,42541.000000,42541.0
mean,103.718085,5.207328e+08,NaN,NaN,NaN,NaN,NaN,NaN,99.430284,34.606550,...,99.483106,2.135197e+11,100.0,0.0,100.0,65535.0,97.424499,8.455872e+11,99.873722,0.0
std,9.892114,1.397802e+09,NaN,NaN,NaN,NaN,NaN,NaN,3.301432,280.074813,...,0.517687,4.767743e+11,0.0,0.0,0.0,0.0,15.758977,1.751869e+11,1.093809,0.0
min,80.000000,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,...,98.000000,2.000000e+00,100.0,0.0,100.0,65535.0,1.000000,4.297720e+09,79.000000,0.0
25%,100.000000,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,0.000000,...,99.000000,1.678414e+06,100.0,0.0,100.0,65535.0,100.000000,7.222055e+11,100.000000,0.0
50%,100.000000,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,0.000000,...,99.000000,7.129478e+08,100.0,0.0,100.0,65535.0,100.000000,8.421974e+11,100.000000,0.0
75%,100.000000,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,2.000000,...,100.000000,2.927546e+11,100.0,0.0,100.0,65535.0,100.000000,9.750210e+11,100.000000,0.0
max,130.000000,4.294967e+09,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,8788.000000,...,100.000000,4.846465e+12,100.0,0.0,100.0,65535.0,100.000000,1.194057e+12,100.000000,0.0


In [127]:
null_cols = ['n_2','r_2','n_3','r_3','n_4','r_4','n_6','r_6','n_7','r_7','n_8','r_8',
             'n_10','r_10','n_11','r_11','n_13','r_13','n_189','r_189','n_191','r_191',
             'n_193','r_193','n_200','r_200','n_204','r_204','n_205','r_205','n_207',
             'r_207','n_211','r_211','n_240','r_240']
mask = (df_clean_desc.loc['std',: ] == 0)
std0_cols = mask[mask == True].index.tolist()

need_del_cols = set(null_cols) | set(std0_cols)
df_clean = df_clean.drop(need_del_cols, 1)
df_clean.shape

(242381, 58)

In [128]:
almost_null = ['n_177', 'r_177', 'n_181', 'r_181', 'n_182', 'r_182', 'n_190', 'r_190', 'r_192', 
               'n_233', 'n_241', 'r_241', 'n_242', 'r_242', 'n_175', 'r_175', 'n_232']
df_clean = df_clean.drop(almost_null, 1)
df_clean.shape


(242381, 41)

#### missing value precessing

- all_neg1: complete with -1

In [129]:
# load checkpoint

# complete with -1
print('df_clean_aug_na_all_neg1 ...')
df_clean_aug_na_all_neg1 = df_clean.fillna(-1)
df_clean_aug_na_all_neg1.to_csv('D:/ssd/df_clean_aug_na_all_neg1.csv',index=None)


df_clean_aug_na_all_neg1 ...


In [130]:
df_clean_aug_na_all_neg1

,disk_id,ds,model,n_1,r_1,n_5,r_5,n_9,r_9,n_12,...,n_195,r_195,n_196,r_196,r_197,r_198,n_199,r_199,r_206,label
10918,406,2018-10-01,MA1,130.0,4.137765e+09,98.0,0.0,100.0,35996.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
345397,406,2018-10-02,MA1,130.0,4.175147e+09,98.0,0.0,100.0,36019.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
80313,406,2018-10-03,MA1,130.0,4.215511e+09,98.0,0.0,100.0,36043.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
242442,406,2018-10-04,MA1,130.0,4.254417e+09,98.0,0.0,100.0,36067.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
114364,406,2018-10-05,MA1,130.0,4.293503e+09,98.0,0.0,100.0,36091.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323799,23851,2018-12-27,MC2,100.0,0.000000e+00,100.0,5.0,100.0,5844.0,100.0,...,100.0,0.0,100.0,5.0,0.0,0.0,100.0,0.0,0.0,no
175712,23851,2018-12-28,MC2,100.0,0.000000e+00,100.0,5.0,100.0,5868.0,100.0,...,100.0,0.0,100.0,5.0,0.0,0.0,100.0,0.0,0.0,no
344020,23851,2018-12-29,MC2,100.0,0.000000e+00,100.0,5.0,100.0,5892.0,100.0,...,100.0,0.0,100.0,5.0,0.0,0.0,100.0,0.0,0.0,no
26818,23851,2018-12-30,MC2,100.0,0.000000e+00,100.0,5.0,100.0,5916.0,100.0,...,100.0,0.0,100.0,5.0,0.0,0.0,100.0,0.0,0.0,no


### split data


In [131]:
nrows = df_clean_aug_na_all_neg1.shape[0]
sub1, sub2 = nrows * 0.3, nrows * 0.2
print(sub1, sub2)

72714.3 48476.200000000004


In [132]:
subset1 = df_clean_aug_na_all_neg1[:round(sub1)]
subset1

,disk_id,ds,model,n_1,r_1,n_5,r_5,n_9,r_9,n_12,...,n_195,r_195,n_196,r_196,r_197,r_198,n_199,r_199,r_206,label
10918,406,2018-10-01,MA1,130.0,4.137765e+09,98.0,0.0,100.0,35996.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
345397,406,2018-10-02,MA1,130.0,4.175147e+09,98.0,0.0,100.0,36019.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
80313,406,2018-10-03,MA1,130.0,4.215511e+09,98.0,0.0,100.0,36043.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
242442,406,2018-10-04,MA1,130.0,4.254417e+09,98.0,0.0,100.0,36067.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
114364,406,2018-10-05,MA1,130.0,4.293503e+09,98.0,0.0,100.0,36091.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396247,19055,2018-12-28,MB1,-1.0,-1.000000e+00,99.0,2.0,97.0,13259.0,99.0,...,200.0,0.0,-1.0,-1.0,0.0,-1.0,100.0,0.0,-1.0,no
360969,19055,2018-12-29,MB1,-1.0,-1.000000e+00,99.0,2.0,97.0,13283.0,99.0,...,200.0,0.0,-1.0,-1.0,0.0,-1.0,100.0,0.0,-1.0,no
99873,19055,2018-12-30,MB1,-1.0,-1.000000e+00,99.0,2.0,97.0,13307.0,99.0,...,200.0,0.0,-1.0,-1.0,0.0,-1.0,100.0,0.0,-1.0,no
117642,19055,2018-12-31,MB1,-1.0,-1.000000e+00,99.0,2.0,97.0,13331.0,99.0,...,200.0,0.0,-1.0,-1.0,0.0,-1.0,100.0,0.0,-1.0,no


In [133]:
subset1.shape

(72714, 41)

In [134]:
subset1['model'].value_counts()

MA2    42547
MA1    19345
MB1    10822
Name: model, dtype: int64

In [135]:
subset1.to_csv('D:/ssd/subset1.csv',index=None)

In [79]:
subset2 = df_clean_aug_na_all_neg1[round(sub1) : round(sub1) + round(sub2)]
subset2['model'].value_counts()
subset2.to_csv('D:/ssd/subset2.csv',index=None)

In [80]:
subset3 = df_clean_aug_na_all_neg1[round(sub1) + round(sub2) : round(sub1) + round(sub2) + round(sub1)]
subset3['model'].value_counts()
subset3.to_csv('D:/ssd/subset3.csv',index=None)

In [81]:
subset4 = df_clean_aug_na_all_neg1[round(sub1) + round(sub2) + round(sub1) : ]
# subset4
subset4['model'].value_counts()
subset4.to_csv('D:/ssd/subset4.csv',index=None)

In [108]:
subset1

,disk_id,ds,model,n_1,r_1,n_5,r_5,n_9,r_9,n_12,...,n_195,r_195,n_196,r_196,r_197,r_198,n_199,r_199,r_206,label
10918,406,2018-10-01,MA1,130.0,4.137765e+09,98.0,0.0,100.0,35996.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
345397,406,2018-10-02,MA1,130.0,4.175147e+09,98.0,0.0,100.0,36019.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
80313,406,2018-10-03,MA1,130.0,4.215511e+09,98.0,0.0,100.0,36043.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
242442,406,2018-10-04,MA1,130.0,4.254417e+09,98.0,0.0,100.0,36067.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
114364,406,2018-10-05,MA1,130.0,4.293503e+09,98.0,0.0,100.0,36091.0,-1.0,...,-1.0,-1.0,100.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100130,25890,2018-10-08,MB1,-1.0,-1.000000e+00,100.0,0.0,97.0,12241.0,99.0,...,200.0,0.0,-1.0,-1.0,0.0,-1.0,100.0,0.0,-1.0,no
8758,25890,2018-10-09,MB1,-1.0,-1.000000e+00,100.0,0.0,97.0,12265.0,99.0,...,200.0,0.0,-1.0,-1.0,0.0,-1.0,100.0,0.0,-1.0,no
382772,25890,2018-10-10,MB1,-1.0,-1.000000e+00,100.0,0.0,97.0,12289.0,99.0,...,200.0,0.0,-1.0,-1.0,0.0,-1.0,100.0,0.0,-1.0,no
335412,26043,2018-10-01,MB1,-1.0,-1.000000e+00,100.0,0.0,97.0,12236.0,99.0,...,200.0,0.0,-1.0,-1.0,0.0,-1.0,100.0,0.0,-1.0,yes
